In [47]:
import re
def extract_bbox_from_text(result_text):
    """
    从生成的文本中提取预测的边界框
    :param result_text: 生成的文本，其中包含边界框信息
    :return: 提取出的边界框，格式为 [xmin, ymin, xmax, ymax]，或空列表如果未找到
    """
    # 定义正则表达式模式以匹配边界框
    pattern = r"\[\s*(0(\.\d+)?|1(\.0+)?)\s*,\s*(0(\.\d+)?|1(\.0+)?)\s*,\s*(0(\.\d+)?|1(\.0+)?)\s*,\s*(0(\.\d+)?|1(\.0+)?)\s*\]"
    match = re.search(pattern, result_text)
    print([float(match.group(i)) for i in range(1, 11, 3)])
    print(match.group(10))
    if match:

        bbox = [float(match.group(i)) for i in range(1, 9, 2)]
        return bbox
    else:
        # 如果未找到匹配的边界框
        print("No bbox found in text")
        return None
rs = "The object mentioned in the expression is the person at the bottom left of the image. The bounding box coordinates of this person are:[0.032,0.444,0.234,0.844]<|eot_id|>"
predicted_bbox = extract_bbox_from_text(rs)

[0.032, 0.444, 0.234, 0.844]
0.844


TypeError: float() argument must be a string or a real number, not 'NoneType'

In [ ]:

def normalize_bbox(self, bbox, w, h):
    # 计算左上和右下坐标
    x, y, width, height = bbox
    xmin = x
    ymin = y
    xmax = x + width
    ymax = y + height

    # 归一化
    xmin_norm = xmin / w
    ymin_norm = ymin / h
    xmax_norm = xmax / w
    ymax_norm = ymax / h

    return [xmin_norm, ymin_norm, xmax_norm, ymax_norm]


In [1]:
# 将合并后的 DataFrame 传递给 RefDataset
from refdataset import RefDataset, IoUEvaluator

import torch
from torch.utils.data import DataLoader
path = '/mnt/pfs-mc0p4k/nlu/team/yuhaofu/eval/refcoco/refer/data/refcoco/testA-00000-of-00001.parquet'
dataset = RefDataset(path)
data_loader = DataLoader(dataset, batch_size=2, shuffle=False)
# Print entire batch structure
for index, batch in enumerate(data_loader):
    annotations, images, true_bboxes, image_paths = batch
    for image, annotation, true_bbox, image_path in zip(images, annotations, true_bboxes, image_paths):
        print(f"Batch {index} - True BBox: {true_bbox}")
        print(true_bboxes.shape)
        break  # 只打印第一项，以检查结构
    break  # 只分析第一个batch

Batch 0 - True BBox: tensor([0.0050, 0.3320, 0.2490, 0.9840])
torch.Size([2, 4])
